In [ ]:
import os 
import glob
import pickle
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from rdkit import Chem
from mordred import Calculator, descriptors
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error  
from sklearn.model_selection import GridSearchCV


In [ ]:
yield_model_path = 'Model_Create_and_Results1/Direct_ary/0_Create_Ground_Truth_Model/ground_truth_model/Ground_Truth_RandomForrest_Mordred.pkl'

with open(yield_model_path, 'rb') as f:
    rf_regressor = pickle.load(f)

# prepare Mordred feature for solvent and bases

In [ ]:
# read in feature list

feature_df = pd.read_csv('Model_Create_and_Results1/Direct_ary/0_Create_Ground_Truth_Model/ground_truth_model/GT_feature_Mordred.csv')['features']
solvent_mordred = pd.read_csv('Model_Create_and_Results1/Direct_ary/0_Create_Ground_Truth_Model/data/solvent_mordred.csv')
base_mordred = pd.read_csv('Model_Create_and_Results1/Direct_ary/0_Create_Ground_Truth_Model/data/base_mordred.csv')

solvent_columns = list(set(feature_df).intersection(solvent_mordred))
solvent_columns.append('solvent_SMILES')
base_columns = list(set(feature_df).intersection(base_mordred))
base_columns.append('base_SMILES')

solvent_ft = solvent_mordred[solvent_columns]
base_ft = base_mordred[base_columns]

ligand_feature = []
for feature in feature_df:
    if 'ligand_' in feature:
        Mordred = feature.split('ligand_')[1]
        ligand_feature.append(Mordred)
        
feature_list = feature_df.tolist()
print(ligand_feature)
print(feature_list)
print(solvent_ft)
print(base_ft)

In [ ]:
# predict the GT values and add them to new dataframe

from sklearn.ensemble import RandomForestRegressor
import pickle

def aryl_yield_predict(df: pd.DataFrame, 
                        aryl_rfr: RandomForestRegressor, 
                        ):
    
    df_yield = df[feature_list]
    # df_score = df[pvk_score_feature_list]

    # return pvk_rfr.predict(df_size), pvk_rfc.predict(df_score), df[pvk_size_feature_list], df[pvk_score_feature_list]
    return aryl_rfr.predict(df_yield)


In [ ]:
cycle_count = ['c2'] # if finished run batch1, the cyce count is c0 for it used the cycle0 surrogate model
methods = ['ABC', 'PSO']
#methods = ['Random/round1', 'Random/round2', 'Random/round3', 'Random/round4', 'Random/round5','Random/round6','Random/round7', 'Random/round8', 'Random/round9', 'Random/round10']
#methods = ['Random/round1', 'Random/round2', 'Random/round3', 'Random/round4', 'Random/round5','Random/round6','Random/round7', 'Random/round8','Random/round10']
methods = ['BO/round1', 'BO/round2', 'BO/round3', 'BO/round4', 'BO/round5',
           'BO/round6', 'BO/round7', 'BO/round8', 'BO/round9', 'BO/round10']
finished_cycle = 2
pattern = r"\d{8}\w+_Report\.csv"
# pvk_size_feature_list = ['Reagent1 (ul)','Reagent2 (ul)','Reagent3 (ul)','Reagent4 (ul)','lab_code','AATS3i','ATSC5Z','AATSC5Z']
# pvk_size_feature_list = ['Reagent1 (ul)','Reagent2 (ul)','Reagent3 (ul)','Reagent4 (ul)','lab_code','ATSC5v', 'AATSC5Z', 'MATS8se']


parent_directory = 'Model_Create_and_Results1/Direct_ary'
preprocessing_for_analysis = os.path.join(parent_directory, '1_Preprocessing_for_Analysis')
make_new_data_predictor = os.path.join(parent_directory, '3_Make_New_Data_Predictor')

cyclen_list = ['Base_SMILES', 'Ligand_SMILES','Solvent_SMILES','Concentration','Temp_C','yield']

for method in methods:
    prediction_folder = os.path.join(preprocessing_for_analysis, method, cycle_count[-1])
    for filename in os.listdir(prediction_folder):
        if re.match(pattern, filename):

            file_path = os.path.join(prediction_folder, filename)
            print(file_path)
            pred_df = pd.read_csv(file_path)
            gt_value = aryl_yield_predict(pred_df, rf_regressor)
            pred_df.rename(columns={'concentration': 'Concentration',
                                    'temperature': 'Temp_C',
                                    'solvent_SMILES': 'Solvent_SMILES',
                                    'base_SMILES': 'Base_SMILES'},
                           inplace=True)
            pred_df['yield'] = gt_value
            pred_df = pred_df[cyclen_list]
            pred_df = pred_df.sort_values(by='yield', ascending=False)

            if len(cycle_count[0]) == 3:
                pred_df.to_csv(os.path.join(make_new_data_predictor, method, f'cycle{int(cycle_count[-1][2])+10}_pred.csv'), index=False)
                print('>10 csv saved')
                df_gt = pd.read_csv(os.path.join(make_new_data_predictor, method, f'cycle{int(cycle_count[-1][2])+10}.csv'))
                df_gt = pd.concat([df_gt, pred_df], axis=0)
                df_gt.to_csv(os.path.join(make_new_data_predictor,method, f'cycle{int(cycle_count[-1][2]) + 11}.csv'), index=False)
            else:
                pred_df.to_csv(os.path.join(make_new_data_predictor, method, f'cycle{int(cycle_count[-1][1])}_pred.csv'), index=False)
                print('csv saved')
                df_gt = pd.read_csv(os.path.join(make_new_data_predictor, method, f'cycle{int(cycle_count[-1][1])}.csv'))
                df_gt = pd.concat([df_gt, pred_df], axis=0)
                df_gt.to_csv(os.path.join(make_new_data_predictor,method, f'cycle{int(cycle_count[-1][1]) + 1}.csv'), index=False)

# Train Predictor

In [ ]:
solvent_mordred = pd.read_csv('Model_Create_and_Results1/Direct_ary/0_Create_Ground_Truth_Model/data/solvent_mordred.csv')
base_mordred = pd.read_csv('Model_Create_and_Results1/Direct_ary/0_Create_Ground_Truth_Model/data/base_mordred.csv')

# prepare the expt + gt data befoe, read differenrt every time
# remeber to change the cycle number !!!!!!!!!!
# cyclen_data = pd.read_csv('/home/ianlee/opt_ian/Model_Create_and_Results1/Direct_ary/3_Make_New_Data_Predictor/PSO/cycle7.csv')

In [ ]:
# get encoder info in X
import sys
sys.path.append('VAE_model/cpu')
from fast_jtnn import *
from rdkit import Chem

# setting VAE params
VAE_path = "VAE_model/model.epoch-39"
vocab_path = "VAE_model/model.epoch-39/smi_vocab-2.txt"

vocab_list = [x.strip("\r\n ") for x in open(vocab_path)]
vocab = Vocab(vocab_list)

hidden_size = 450
latent_size = 32
depthT = 20
depthG = 3

device = torch.device("cpu")
print(torch.cuda.device_count())

VAE = JTNNVAE(vocab, hidden_size, latent_size, depthT, depthG)
VAE.load_state_dict(torch.load(VAE_path,map_location=(device)))
VAE.cpu()
VAE.eval()

X = {}
Y = {}

for method in methods:
    #obtain SMILES and latent vector
    cyclen_data = pd.DataFrame(pd.read_csv(os.path.join(make_new_data_predictor, f'{method}/cycle{finished_cycle+1}.csv')))

    merged_data = pd.merge(cyclen_data, base_mordred, left_on='Base_SMILES', right_on='base_SMILES', how='left')
    merged_data = pd.merge(merged_data, solvent_mordred, left_on='Solvent_SMILES', right_on='solvent_SMILES', how='left')


    merged_data = merged_data.drop(columns=['Base_SMILES', 'Solvent_SMILES','base_SMILES', 'solvent_SMILES','Unnamed: 0', 'yield'])

 
    
    data_smi = list(merged_data['Ligand_SMILES'])
        
    #print(data_smi)

    latent = VAE.encode_latent_mean(data_smi)
    latent = latent.detach().cpu().numpy() #latent is a huge numpy array which need to be concat with features


    merged_data = merged_data.drop(columns=['Ligand_SMILES'])

    # We need to assume that we do not know anything about the solvent and base -- cannot filter the feautres
    # X_train = np.concatenate((latent, merged_data[predictor_feature['Feature']]), axis=1)
    X[method] = np.concatenate((latent, merged_data), axis=1)
    print(X[method].shape)
    Y[method] = cyclen_data['yield']
    # total = merged_data[predictor_feature['Feature'].tolist() + ['yield']]
    # total.to_csv('total.csv')

In [ ]:
from sklearn.model_selection import cross_val_score

for method in methods:
    # assign the new folder here
    xgb_folder = os.path.join(make_new_data_predictor, f'{method}/cycle{finished_cycle+1}')

    parameters = {
        'n_estimators': [90, 100, 110],
        'max_depth': [5, 6, 7],
        'learning_rate': [0.2, 0.3, 0.4],
        'subsample': [0.8, 0.9, 1.0],  

    }

    model = XGBRegressor()
    grid_search = GridSearchCV(model, parameters, cv=5)
    grid_search.fit(X[method], Y[method])
    best_model = grid_search.best_estimator_

    y_pred = best_model.predict(X[method])
    mse = mean_squared_error(Y[method], y_pred)
    mean = np.mean(Y[method])  
    std = np.std(Y[method])

    r2_scores = cross_val_score(model, X[method], y_pred, cv=5, scoring='r2')

    plt.figure(figsize=(8, 8))
    plt.scatter(Y[method], y_pred, alpha=0.3)
    plt.plot([Y[method].min(), Y[method].max()], [Y[method].min(), Y[method].max()], 'k--')
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.title('Parity plot')
    plt.show()

    print("MSE between Train and Prediction: ", mse)
    print("Mean of Training Data y: ", mean)
    print("Std Dev of Training Data y: ", std)
    print("cross validation r2 scores: ", r2_scores)
    print("mean cross validation r2 scores: ", np.mean(r2_scores))
    results = pd.DataFrame(grid_search.cv_results_)
    results.to_csv('opt_result.csv')
    results = results.sort_values("mean_test_score", ascending=False)
    results = results[["mean_test_score", "params"]]
    results = results[:10]

    count = 0

    for i, row in results.iterrows():
        hyp = row[1]

        xgb = XGBRegressor(**hyp)
        xgb.fit(X[method], Y[method])

        with open(os.path.join(xgb_folder,f"ary_yield_xgboost{count}_c{int(cycle_count[-1][1])}.pkl"), "wb") as f:
            pickle.dump(xgb, f)
        count += 1

In [ ]:
results